In [1]:
spark

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
3,application_1748219621515_0004,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
sc

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<SparkContext master=yarn appName=livy-session-3>

In [ ]:
import pandas as pd
import umap
import numpy as np
from sklearn.pipeline import Pipeline as sklearn_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, davies_bouldin_score
from sklearn.manifold import trustworthiness

In [ ]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [3]:
from pyspark.sql import SparkSession, Row, Window
from pyspark.sql.functions import col, countDistinct, regexp_replace, split, trim, count, when, rand, row_number, vector_to_array
from pyspark.sql.types import DoubleType, DateType, FloatType
from pyspark.ml import Pipeline as pyspark_pipeline
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler
from pyspark.ml.evaluation import MulticlassClassificationEvaluator, BinaryClassificationEvaluator, ClusteringEvaluator
from pyspark.ml.classification import DecisionTreeClassifier,LogisticRegression
from pyspark.ml.clustering import KMeans
from pyspark.ml.linalg import Vectors, VectorUDT
from pyspark.sql import functions as F
import re

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
import numba
numba.config.CACHE_DIR = '/tmp/numba_cache'  # or any writable path
numba.config.DISABLE_JIT = False  # keep JIT but avoid default caching errors

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
spark = SparkSession.builder.appName("PMOProjectML").getOrCreate()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
df = spark.read.parquet("s3://trusted-proyecto-integrador/pmo_data/")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
df.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---+--------------------+--------------------+--------------------+----------+----------+--------------------+--------+------+----------------+----------------+-------------------+--------+--------------------+-------------------+---------------+-------------------+-------------+----------------+---------------+--------------------+-----------------------+------------------------+-------------+----------+-----------------+------------+---------------+-------------+-----------+--------------------+----------------+--------------+---------+-----------------+----------+---------------+-----------+------------+---------+
| c0|          project_id|      reporting_name|        project_name| createdon|   updated|               phase|duration|effort|effort_completed|effort_remaining|             finish|progress|  project_manager_id|              start|task_start_date|    project_manager|business_unit|project_priority|      portfolio|proj_issuehealthname|proj_schedulehealthname|proj_prioritiza

In [10]:
df.describe().show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+-----------------+--------------------+--------------------+--------------------+--------------------+-----------------+------------------+------------------+-----------------+-------------------+--------------------+----------------+-------------+-------------------+-----------------+--------------------+-----------------------+------------------------+-------------+--------------------+------------------+-------------+-------+-----+------------------+--------------+------------------+------------------+------------------+-------------------+-------------------+-------------------+-----------------+
|summary|               c0|          project_id|      reporting_name|        project_name|               phase|         duration|            effort|  effort_completed| effort_remaining|           progress|  project_manager_id| project_manager|business_unit|   project_priority|        portfolio|proj_issuehealthname|proj_schedulehealthname|proj_prioritizationscore|project_trend|    

In [11]:
def get_column_types(df):
    schema = df.dtypes
    dict_types = {'string': [], 'date': [], 'numeric': []}
    for col_name, dtype in schema:
        if dtype in ['string']:
            dict_types['string'].append(col_name)
        elif dtype in ['date']:
            dict_types['date'].append(col_name)
        else:
            dict_types['numeric'].append(col_name)
    return dict_types

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
column_types = get_column_types(df)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [13]:
column_types

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

{'string': ['project_id', 'reporting_name', 'project_name', 'phase', 'project_manager_id', 'project_manager', 'business_unit', 'portfolio', 'proj_issuehealthname', 'proj_schedulehealthname', 'project_trend', 'client', 'eho_completed', 'country', 'site', 'line_of_business', 'overall_health'], 'date': ['createdon', 'updated', 'task_start_date'], 'numeric': ['c0', 'duration', 'effort', 'effort_completed', 'effort_remaining', 'finish', 'progress', 'start', 'project_priority', 'proj_prioritizationscore', 'commercial', 'training_billable', 'total_headcount', 'num_risks', 'avg_risk_exposure', 'num_issues', 'avg_issue_score', 'num_changes', 'change_score', 'num_tasks']}

In [14]:
string_cols = column_types['string']

unique_counts = {
    col_name: df.select(countDistinct(col(col_name))).collect()[0][0]
    for col_name in string_cols
}
sorted_unique_counts = dict(sorted(unique_counts.items(), key=lambda item: item[1], reverse=True))

for col_name, count in sorted_unique_counts.items():
    print(f"{col_name}: {count}")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

project_id: 2117
project_name: 2114
reporting_name: 1831
site: 1064
client: 639
line_of_business: 597
project_manager_id: 241
project_manager: 240
country: 65
portfolio: 9
phase: 7
proj_issuehealthname: 3
proj_schedulehealthname: 3
project_trend: 3
overall_health: 3
business_unit: 2
eho_completed: 2

In [15]:
df.select("commercial").show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+
|commercial|
+----------+
|     false|
|     false|
|     false|
|     false|
|     false|
|      true|
|     false|
|     false|
|     false|
|     false|
|     false|
|     false|
|     false|
|      true|
|     false|
|     false|
|     false|
|     false|
|     false|
|     false|
+----------+
only showing top 20 rows

In [16]:
cols_to_encode = ['phase',
 'business_unit',
 'portfolio',
 'proj_issuehealthname',
 'proj_schedulehealthname',
 'project_trend',
 'eho_completed',
 'overall_health']

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [17]:
indexers = [StringIndexer(inputCol=col, outputCol=col+"_idx", handleInvalid='keep') for col in cols_to_encode]
encoders = [OneHotEncoder(inputCol=col+"_idx", outputCol=col+"_ohe") for col in cols_to_encode]
pipeline = pyspark_pipeline(stages=indexers + encoders)
df_encoded_model = pipeline.fit(df)
df_encoded = df_encoded_model.transform(df)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [18]:
df_encoded.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---+--------------------+--------------------+--------------------+----------+----------+--------------------+--------+------+----------------+----------------+-------------------+--------+--------------------+-------------------+---------------+-------------------+-------------+----------------+---------------+--------------------+-----------------------+------------------------+-------------+----------+-----------------+------------+---------------+-------------+-----------+--------------------+----------------+--------------+---------+-----------------+----------+---------------+-----------+------------+---------+---------+-----------------+-------------+------------------------+---------------------------+-----------------+-----------------+------------------+-------------+-----------------+-------------+------------------------+---------------------------+-----------------+-----------------+------------------+
| c0|          project_id|      reporting_name|        project_name| c

In [19]:
df_final = df_encoded.drop('VOC Score', 'VOC Score IT')
df_pd = df_final.toPandas()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [20]:
k_means_columns = ['project_priority', 'avg_risk_exposure', 'total_headcount', 'num_tasks', 'duration']

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [56]:
X = df_pd[k_means_columns]
indices = np.arange(len(df_pd))
X_train, X_test,indices_train,indices_test = train_test_split(X, indices, test_size=0.3, random_state=42)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [55]:
df_pd.columns

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Index(['c0', 'project_id', 'reporting_name', 'project_name', 'createdon',
       'updated', 'phase', 'duration', 'effort', 'effort_completed',
       'effort_remaining', 'finish', 'progress', 'project_manager_id', 'start',
       'task_start_date', 'project_manager', 'business_unit',
       'project_priority', 'portfolio', 'proj_issuehealthname',
       'proj_schedulehealthname', 'proj_prioritizationscore', 'project_trend',
       'commercial', 'training_billable', 'client', 'total_headcount',
       'eho_completed', 'country', 'site', 'line_of_business',
       'overall_health', 'num_risks', 'avg_risk_exposure', 'num_issues',
       'avg_issue_score', 'num_changes', 'change_score', 'num_tasks',
       'phase_idx', 'business_unit_idx', 'portfolio_idx',
       'proj_issuehealthname_idx', 'proj_schedulehealthname_idx',
       'project_trend_idx', 'eho_completed_idx', 'overall_health_idx',
       'phase_ohe', 'business_unit_ohe', 'portfolio_ohe',
       'proj_issuehealthname_ohe', 'proj_s

In [22]:
preprocessing_pipeline = sklearn_pipeline(steps=[
    ('scaler', StandardScaler()),
    ('umap', umap.UMAP(n_components=2, random_state=42))
])

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [23]:
# Pipeline en Train (Fit & Transform)
X_train_umap = preprocessing_pipeline.fit_transform(X_train)

# Pipeline en Test (Transform)
X_test_umap = preprocessing_pipeline.transform(X_test)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

/usr/local/lib64/python3.9/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.9/site-packages/umap/umap_.py:1952: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(
/usr/local/lib64/python3.9/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(

In [24]:
score = trustworthiness(X_train, X_train_umap, n_neighbors=2)
print(f"Trustworthiness of UMAP embedding: {score:.4f}")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Trustworthiness of UMAP embedding: 0.8997

In [25]:
train_df = spark.createDataFrame(X_train_umap)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [26]:
test_df = spark.createDataFrame(X_test_umap)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [27]:
def inputar(df):
    df_new = df
    numeric_cols = [f.name for f in df_new.schema.fields if str(f.dataType) in ['DoubleType', 'IntegerType', 'FloatType', 'LongType', 'ShortType']]
    medians = {
      col_name: df_new.approxQuantile(col_name, [0.5], 0.0)[0]
      for col_name in numeric_cols
    }

    for col_name in numeric_cols:
        median_value = medians[col_name]
        df_new = df_new.withColumn(
            col_name,
            when(col(col_name).isNull(), median_value).otherwise(col(col_name))
        )

    return df_new

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [28]:
train_df = inputar(train_df)
test_df = inputar(test_df)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [29]:
assembler = VectorAssembler(inputCols=["_1", "_2"], outputCol="features_umap")
train_df_assembled = assembler.transform(train_df)
test_df_assembled = assembler.transform(test_df)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [30]:
kmeans = KMeans(k=2, seed=42, featuresCol="features_umap", predictionCol="cluster")
kmeans_model = kmeans.fit(train_df_assembled)
train_clustered = kmeans_model.transform(train_df_assembled)
train_clustered.select("features_umap", "cluster").show(truncate=False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------------------------------------+-------+
|features_umap                           |cluster|
+----------------------------------------+-------+
|[11.923763275146484,14.172167778015137] |0      |
|[6.886080265045166,1.4162100553512573]  |1      |
|[11.567461013793945,-2.9020538330078125]|1      |
|[7.853046417236328,26.158838272094727]  |0      |
|[11.959892272949219,-0.8185890913009644]|1      |
|[10.225910186767578,9.527958869934082]  |0      |
|[6.514909267425537,0.36157605051994324] |1      |
|[6.545828819274902,4.489383697509766]   |0      |
|[3.5333456993103027,2.846126079559326]  |0      |
|[11.215930938720703,-1.011230707168579] |1      |
|[6.092734336853027,8.322957992553711]   |0      |
|[12.871318817138672,1.9278618097305298] |1      |
|[10.333907127380371,0.47693297266960144]|1      |
|[2.7984323501586914,3.4681520462036133] |0      |
|[8.304429054260254,8.412611961364746]   |0      |
|[10.060134887695312,1.4578202962875366] |1      |
|[11.021610260009766,-11.705299

In [31]:
pandas_data = train_clustered.select("features_umap", "cluster").toPandas()
pandas_data[['PC1', 'PC2']] = pandas_data['features_umap'].apply(lambda x: pd.Series(x.toArray()))
pandas_data['cluster'] = pandas_data['cluster'].astype(int)
centroids = kmeans_model.clusterCenters()
centroids = np.array(centroids)


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [32]:

evaluator = ClusteringEvaluator(
    predictionCol='cluster',
    featuresCol='features_umap',
    metricName='silhouette',
    distanceMeasure='squaredEuclidean'
)

sil_score = evaluator.evaluate(train_clustered)
print(f"Silhouette Score: {sil_score:.4f}")


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Silhouette Score: 0.5627

In [33]:
pdf = train_clustered.select("features_umap", "cluster").toPandas()
pdf[['C1', 'C2']] = pdf['features_umap'].apply(lambda x: pd.Series(x.toArray()))
labels = pdf['cluster']
X_train_pca = pdf[['C1', 'C2']]

db_index = davies_bouldin_score(X_train_pca, labels)
print(f"Davies-Bouldin Index: {db_index:.4f}")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Davies-Bouldin Index: 0.8739

In [34]:

labeled_df = train_clustered.withColumnRenamed("cluster", "label")

k = 5

models = {
    'Logistic Regression': LogisticRegression(featuresCol='features_umap', labelCol='label', maxIter=10),
    'Decision Tree': DecisionTreeClassifier(featuresCol='features_umap', labelCol='label', seed=42)
}
metrics_vec = []
metrics_cv_kfold = {}

labeled_df = labeled_df.withColumn("rand", rand())
window = Window.orderBy("rand")
labeled_df = labeled_df.withColumn("row_idx", row_number().over(window))
total_rows = labeled_df.count()
fold_size = total_rows // k
labeled_df = labeled_df.withColumn("fold", ((col("row_idx") - 1) / fold_size).cast("int"))

accuracy_eval = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
precision_eval = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="weightedPrecision")
recall_eval = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="weightedRecall")
auc_eval = BinaryClassificationEvaluator(labelCol="label", rawPredictionCol="rawPrediction", metricName="areaUnderROC")


for model_name, model in models.items():
    accuracies, precisions, recalls, aucs = [], [], [], []
    for i in range(k):
            train_fold = labeled_df.filter(col("fold") != i)
            val_fold = labeled_df.filter(col("fold") == i)
            trained_model = model.fit(train_fold)
            predictions = trained_model.transform(val_fold)

            acc = accuracy_eval.evaluate(predictions)
            prec = precision_eval.evaluate(predictions)
            rec = recall_eval.evaluate(predictions)

            try:
                    auc = auc_eval.evaluate(predictions)
            except:
                    auc = float('nan')

            accuracies.append(acc)
            precisions.append(prec)
            recalls.append(rec)
            aucs.append(auc)

    mean_acc = np.nanmean(accuracies)
    mean_prec = np.nanmean(precisions)
    mean_rec = np.nanmean(recalls)
    mean_auc = np.nanmean(aucs)
    metrics_cv_kfold[model_name] = {
            'Accuracy': mean_acc,
            'Precision': mean_prec,
            'Recall': mean_rec,
            'AUC': mean_auc
    }

    metrics_val = [model_name, mean_acc, mean_prec,mean_rec,mean_auc]
    metrics_vec.append(metrics_val)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [35]:
df_metrics = pd.DataFrame(metrics_vec, columns=['Modelo', 'Accuracy Mean (K folds)', 'Precision Mean (K folds)', 'Recall Mean (K folds)', 'ROC AUC Mean (K folds)'])
df_metrics.set_index('Modelo', inplace=True)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [36]:
df_metrics.head()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

                     Accuracy Mean (K folds)  Precision Mean (K folds)  \
Modelo                                                                   
Logistic Regression                 0.997297                  0.997305   
Decision Tree                       0.995946                  0.995978   

                     Recall Mean (K folds)  ROC AUC Mean (K folds)  
Modelo                                                              
Logistic Regression               0.997297                0.999972  
Decision Tree                     0.995946                0.999171

In [37]:
best_model = models['Decision Tree']
trained_model = best_model.fit(labeled_df)
test_clustered = kmeans_model.transform(test_df_assembled).withColumnRenamed("prediction", "label")
test_predictions = trained_model.transform(test_clustered)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [38]:
pdf = test_predictions.select("cluster", "prediction").toPandas()
print(classification_report(pdf["cluster"], pdf["prediction"], digits=4))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

              precision    recall  f1-score   support

           0     0.9947    0.9973    0.9960       376
           1     0.9961    0.9923    0.9942       260

    accuracy                         0.9953       636
   macro avg     0.9954    0.9948    0.9951       636
weighted avg     0.9953    0.9953    0.9953       636

In [39]:

cm = confusion_matrix(pdf["cluster"], pdf["prediction"])

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [40]:
cm

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

array([[375,   1],
       [  2, 258]])

In [67]:
from pyspark.sql.functions import monotonically_increasing_id

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [84]:
df_original_pd = df.toPandas()
df1 = labeled_df.select("_1", "_2", 'label')
df1 = df1.withColumnRenamed("_1", "umap_1")
df1 = df1.withColumnRenamed("_2", "umap_2")
df1 = df1.withColumnRenamed("label", "cluster")

df1_cols = spark.createDataFrame(df_original_pd.iloc[indices_train,:])
df1_with_index = df1.withColumn("row_id", monotonically_increasing_id())
df1_cols_with_index = df1_cols.withColumn("row_id", monotonically_increasing_id())
old_points_df = df1_cols_with_index.join(df1_with_index, on="row_id").drop("row_id")


df2 = test_predictions.select("_1", "_2", 'cluster', 'prediction')
df2 = df2.withColumnRenamed("_1", "umap_1")
df2 = df2.withColumnRenamed("_2", "umap_2")
df2 = df2.withColumnRenamed("label", "cluster")
df2_cols = spark.createDataFrame(df_original_pd.iloc[indices_test,:])
df2_with_index = df2.withColumn("row_id", monotonically_increasing_id())
df2_cols_with_index = df2_cols.withColumn("row_id", monotonically_increasing_id())
new_points_df = df2_cols_with_index.join( df2_with_index, on="row_id").drop("row_id")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [85]:


# Añadir vector con características
feature_cols = ['project_priority', 'avg_risk_exposure', 'total_headcount', 'num_tasks', 'duration']

def to_vector_udf(cols):
    return F.udf(lambda row: Vectors.dense(row), VectorUDT())(F.array(*[F.col(c) for c in cols]))

old_points_df = old_points_df.withColumn("features", to_vector_udf(feature_cols))
new_points_df = new_points_df.withColumn("features", to_vector_udf(feature_cols))

# Join según el cluster al que fue asignado
joined_df = new_points_df.alias("new").join(
    old_points_df.alias("old"),
    F.col("new.prediction") == F.col("old.cluster")
)

# Computar distancias
def euclidean_distance(v1, v2):
    return float(v1.squared_distance(v2)) ** 0.5

euclidean_distance_udf = F.udf(euclidean_distance, FloatType())

joined_df = joined_df.withColumn(
    "distance",
    euclidean_distance_udf(F.col("new.features"), F.col("old.features"))
)

# Hacer un ranking de proyectos similares según la distancia y seleccionar el top 3
window_spec = Window.partitionBy("new.project_id").orderBy("distance")

top_k_df = joined_df.withColumn("rank", F.row_number().over(window_spec)) \
                    .filter(F.col("rank") <= 3) \
                    .groupBy("new.project_id") \
                    .agg(F.collect_list("old.project_id").alias("top_3_neighbors"))

# Join para obtener un dataframe con los proyectos más similares de la clase a la que fue asignada el nuevo proyecto
new_points_with_neighbors = new_points_df.join(
    top_k_df,
    on="project_id",
    how="left"
)


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [76]:
merged_df1.columns

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['c0', 'project_id', 'reporting_name', 'project_name', 'createdon', 'updated', 'phase', 'duration', 'effort', 'effort_completed', 'effort_remaining', 'finish', 'progress', 'project_manager_id', 'start', 'task_start_date', 'project_manager', 'business_unit', 'project_priority', 'portfolio', 'proj_issuehealthname', 'proj_schedulehealthname', 'proj_prioritizationscore', 'project_trend', 'commercial', 'training_billable', 'client', 'total_headcount', 'eho_completed', 'country', 'site', 'line_of_business', 'overall_health', 'num_risks', 'avg_risk_exposure', 'num_issues', 'avg_issue_score', 'num_changes', 'change_score', 'num_tasks', 'phase_idx', 'business_unit_idx', 'portfolio_idx', 'proj_issuehealthname_idx', 'proj_schedulehealthname_idx', 'project_trend_idx', 'eho_completed_idx', 'overall_health_idx', 'phase_ohe', 'business_unit_ohe', 'portfolio_ohe', 'proj_issuehealthname_ohe', 'proj_schedulehealthname_ohe', 'project_trend_ohe', 'eho_completed_ohe', 'overall_health_ohe', 'umap_1', 'umap_

In [71]:
merged_df2.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

636

In [87]:
new_points_with_neighbors.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+----+--------------------+--------------------+----------+----------+--------------------+--------+------+----------------+----------------+-------------------+--------+--------------------+-------------------+---------------+--------------------+-------------+----------------+------------------+--------------------+-----------------------+------------------------+-------------+----------+-----------------+-------------+---------------+-------------+--------------------+--------------------+----------------+--------------+---------+-----------------+----------+---------------+-----------+------------+---------+----------+------------+-------+----------+--------------------+--------------------+
|          project_id|  c0|      reporting_name|        project_name| createdon|   updated|               phase|duration|effort|effort_completed|effort_remaining|             finish|progress|  project_manager_id|              start|task_start_date|     project_manager|busin

In [88]:
new_points_with_neighbors.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

636

In [86]:
relationship_df = new_points_with_neighbors.select(
    F.col("project_id").alias("id_project_new_point"),
    F.explode("top_3_neighbors").alias("id_project_similar_point")
)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [83]:
relationship_df.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+------------------------+
|id_project_new_point|id_project_similar_point|
+--------------------+------------------------+
|00761285-CECE-464...|    CF8FEBD5-6F00-446...|
|00761285-CECE-464...|    C6D891CE-AC59-41B...|
|00761285-CECE-464...|    30685825-3022-48C...|
|00C6D5C0-55A0-EF1...|    4CA7DCA2-4B04-F01...|
|00C6D5C0-55A0-EF1...|    5A7F88BF-F607-F01...|
|00C6D5C0-55A0-EF1...|    ABABB0F8-5A92-EF1...|
|00EEB192-C8F2-EE1...|    9772D0B2-5C17-EF1...|
|00EEB192-C8F2-EE1...|    A61F8A20-4CD0-EE1...|
|00EEB192-C8F2-EE1...|    2CBEB543-0021-EE1...|
|01707E98-624E-4EA...|    3467C167-74A8-ED1...|
|01707E98-624E-4EA...|    2AB3A54E-0CFB-EE1...|
|01707E98-624E-4EA...|    32CE1D3F-0DFB-EE1...|
|017F7296-6173-EE1...|    1C2F3415-266C-EF1...|
|017F7296-6173-EE1...|    0291D0BF-5F6F-EF1...|
|017F7296-6173-EE1...|    458D9B33-A867-EE1...|
|022D1315-6F1E-416...|    E55D8C41-EDE4-EE1...|
|022D1315-6F1E-416...|    B47C6E99-4E61-43C...|
|022D1315-6F1E-416...|    8B21E43F-FEE5-

In [91]:
s3_output_path = "s3://refined-proyecto-integrador/train_pmo_data"
old_points_df.write.mode("overwrite").parquet(s3_output_path)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [92]:
s3_output_path = "s3://refined-proyecto-integrador/predicted_pmo_data"
new_points_with_neighbors.write.mode("overwrite").parquet(s3_output_path)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [93]:
s3_output_path = "s3://refined-proyecto-integrador/relationships_pmo_data"
relationship_df.write.mode("overwrite").parquet(s3_output_path)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…